In [1]:
!nvidia-smi

Thu Mar 19 11:21:29 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 435.21       Driver Version: 435.21       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0  On |                  N/A |
|  0%   35C    P5    22W / 250W |   1227MiB / 11176MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [1]:
import os
import re
import PIL
import sys
import json
import time
import timm
import math
import copy
import torch
import pickle
import geffnet
import logging
import fnmatch
import argparse
import torchvision
import numpy as np
%matplotlib inline
import pandas as pd
import seaborn as sns
import torch.nn as nn
from PIL import Image
from pathlib import Path
from copy import deepcopy
from sklearn import metrics
import torch.optim as optim
from datetime import datetime
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.utils.data as data
from geffnet import create_model
from torch.autograd import Variable
from tqdm import tqdm, tqdm_notebook
from torch.optim import lr_scheduler
import torch.utils.model_zoo as model_zoo
from efficientnet_pytorch import EfficientNet
%config InlineBackend.figure_format = 'retina'
from collections import OrderedDict, defaultdict
from torchvision import transforms, models, datasets
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import confusion_matrix,accuracy_score
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
data_dir = '/home/linh/Downloads/DeepWeeds/'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'

# Define your transforms for the training and testing sets
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomRotation(30),
        transforms.RandomResizedCrop(224),
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], 
                             [0.229, 0.224, 0.225])
    ])
}

# Load the datasets with ImageFolder
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val']}

# Using the image datasets and the trainforms, define the data_loader
# batch_size = 64 for EfficientNet from B0 - B3
# batch_size = 32 for EfficientNet B4, B5
# batch_size = 16 for EfficientNet_B6
# batch_size = 8 for EfficientNet_B7
# batch_size = 32 for MixNet_s
batch_size = 16
data_loader = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size,
                                             shuffle=True, num_workers=4, pin_memory = True)
              for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}

class_names = image_datasets['train'].classes

print(dataset_sizes)
print(device)

"""# Label mapping
with open('/home/linh/Downloads/Derma/cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)"""

''' 
f = open('/home/linh/Downloads/Derma/classes.txt','r')
cat_to_name = f.read()
print(cat_to_name)
f.close()
'''
### we get the class_to_index in the data_Set but what we really need is the cat_to_names  so we will create
_ = image_datasets['train'].class_to_idx
cat_to_name = {_[i]: i for i in list(_.keys())}

    
# Run this to test the data loader
images, labels = next(iter(data_loader['val']))
images.size()

{'train': 14007, 'val': 3502}
cuda:0


torch.Size([16, 3, 224, 224])

In [3]:
#model = EfficientNet.from_pretrained('efficientnet-b3')
#model = timm.create_model('tf_efficientnet_b6', pretrained = True)
#from efficientnet_pytorch import EfficientNet
#model = EfficientNet.from_pretrained('efficientnet-b6')
model = create_model('tf_efficientnet_lite4', pretrained=True)
# Create classifier
for param in model.parameters():
    param.requires_grad = True
n_classes = 9
model.classifier = nn.Linear(model.classifier.in_features, n_classes)

criterion = nn.CrossEntropyLoss()
#optimizer = Nadam(model.parameters(), lr=0.001)
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0.0001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


In [12]:
_ = image_datasets['train'].class_to_idx
cat_to_name = {_[i]: i for i in list(_.keys())}
cat_to_name
class_names = image_datasets['train'].classes
class_names

['Chinee apple',
 'Lantana',
 'Negative',
 'Parkinsonia',
 'Parthenium',
 'Prickly acacia',
 'Rubber vine',
 'Siam weed',
 'Snake weed']

In [13]:
model.to(device)

def train_model(model, criterion, optimizer, scheduler, num_epochs=200, checkpoint = None):
    since = time.time()

    if checkpoint is None:
        best_model_wts = copy.deepcopy(model.state_dict())
        best_loss = math.inf
        best_acc = 0.
    else:
        print(f'Val loss: {checkpoint["best_val_loss"]}, Val accuracy: {checkpoint["best_val_accuracy"]}')
        model.load_state_dict(checkpoint['model_state_dict'])
        best_model_wts = copy.deepcopy(model.state_dict())
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        best_loss = checkpoint['best_val_loss']
        best_acc = checkpoint['best_val_accuracy']

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for i, (inputs, labels) in enumerate(data_loader[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()
                
                if i % 1000 == 999:
                    print('[%d, %d] loss: %.8f' % 
                          (epoch + 1, i, running_loss / (i * inputs.size(0))))

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':                
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':                
                scheduler.step()
                
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.8f} Acc: {:.8f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_loss < best_loss:
                print(f'New best model found!')
                print(f'New record loss: {epoch_loss}, previous record loss: {best_loss}')
                best_loss = epoch_loss
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                torch.save({'model_state_dict': model.state_dict(),
                            'optimizer_state_dict': optimizer.state_dict(),
                            'best_val_loss': best_loss,
                            'best_val_accuracy': best_acc,
                            'scheduler_state_dict' : scheduler.state_dict(),
                            }, 
                            CHECK_POINT_PATH
                            )
                print(f'New record loss is SAVED: {epoch_loss}')
 
        print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:.8f} Best val loss: {:.8f}'.format(best_acc, best_loss))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, best_loss, best_acc

In [ ]:
CHECK_POINT_PATH = '/home/linh/Downloads/DeepWeeds/weights/EfficientNet_Lite4_ADAM.pth'
try:
    checkpoint = torch.load(CHECK_POINT_PATH)
    print("checkpoint loaded")
except:
    checkpoint = None
    print("checkpoint not found")
if checkpoint == None:
    CHECK_POINT_PATH = CHECK_POINT_PATH
model, best_val_loss, best_val_acc = train_model(model,
                                                 criterion,
                                                 optimizer,
                                                 scheduler,
                                                 num_epochs = 100,
                                                 checkpoint = torch.load(CHECK_POINT_PATH)
                                                 ) 
                                                
torch.save({'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'best_val_loss': best_val_loss,
            'best_val_accuracy': best_val_acc,
            'scheduler_state_dict': scheduler.state_dict(),
            }, CHECK_POINT_PATH)

checkpoint loaded
Val loss: 0.09322643780425104, Val accuracy: 0.9691695906432749
Epoch 0/99
----------
[1, 999] loss: 0.21498878
[1, 1999] loss: 0.21604584
[1, 2999] loss: 0.21728260
[1, 3999] loss: 0.21537541
train Loss: 0.21522474 Acc: 0.92673684
[1, 999] loss: 0.09245275
val Loss: 0.09273952 Acc: 0.96987135
New best model found!
New record loss: 0.09273952202559911, previous record loss: 0.09322643780425104
New record loss is SAVED: 0.09273952202559911

Epoch 1/99
----------
[2, 999] loss: 0.21608521
[2, 1999] loss: 0.20994701
[2, 2999] loss: 0.21205027
[2, 3999] loss: 0.21395211
train Loss: 0.21392724 Acc: 0.92647173
[2, 999] loss: 0.09632084
val Loss: 0.09476769 Acc: 0.96893567

Epoch 2/99
----------
[3, 999] loss: 0.21729812
[3, 1999] loss: 0.21565302
[3, 2999] loss: 0.21457699
[3, 3999] loss: 0.21547103
train Loss: 0.21545175 Acc: 0.92764133
[3, 999] loss: 0.09482938
val Loss: 0.09519588 Acc: 0.96898246

Epoch 3/99
----------
[4, 999] loss: 0.21647196
[4, 1999] loss: 0.21168381